In [1]:
#################################################################
# Code      Mass_Suite_NNmodel_Keras v1.2
# Version   1.2
# Date      2020-04-28
# Author    Bowei Zhang, boweiz@uw.edu
# Copyright Bowei Zhang, University of Washington, Seattle, WA

# Note      This code can train a Neural Network using Keras
#           Python V3.7.4
#           Tensorflow V2.0.0
#################################################################

In [120]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
import scipy.interpolate
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from tensorflow import keras

In [121]:
from tensorflow import keras
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn
import os
from sklearn.model_selection import train_test_split
import scipy.interpolate

In [122]:
input_file  = "upsampled-modelingdata.csv"
data = pd.read_csv(input_file)

#Input and output columns in csv file
inputs = ["peak width in min", "half intensity width in min", "left width", "right width", 
          "assymetric factor", "integration", "sn", "hw", "ab", "peak height", "ma", "mb", 
          "broad rate", "skewness", "variance"]
output = "label"

In [165]:
#Normalize the data.
#0 for Not Normalizing 
#Option 1:  x_i = (x_i - x_mean)/(std)
#Option 2:  x_i = (x_i - x_min)/(x_max - x_min)
#Option 3:  x_i = 2(x_i - x_min)/(x_max - x_min)-1
normalize = 1                                

#Number estimators
num_of_estimators = 100

#Number of models
num = 3

#Read data and remove unrealistic data
df = pd.read_csv(input_file)

#The total number of data
data_number = len(df)



In [166]:
#Training and validation sets
x = data[inputs]
y = data[output]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

trained_models = {}

# for output in outputs:
   
model = RandomForestClassifier(n_estimators=num_of_estimators)
models = [model]
for i in range(num-1):
    models.append(sklearn.base.clone(models[0],safe=True))

#Fit the model
for i in range(num):
    models[i].fit(x_train, y_train)
    print('finished',i+1,'out of',num)

trained_models[output] = models

finished 1 out of 3
finished 2 out of 3
finished 3 out of 3


In [167]:
#Evaluate the model
model_evals = {}
best_models = {}

# for output in outputs:
best_model_num = 0
max_accuracy = 0
for i in range(num):
    y_pred=models[i].predict(x_test)
    score = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy for validation set {} is {:.3f}".format(i+1, metrics.accuracy_score(y_test, y_pred)))
    if (score > max_accuracy):
        best_model_num = i
        max_accuracy = score

best_models[output] = models[best_model_num]

print("The highest accuracy is {:.3f}, from model {:d}".format(max_accuracy, best_model_num+1))
#Save the model with the best performance
time = datetime.now().timetuple()[1:6]
#os.makedirs('./models', exist_ok=True)

Accuracy for validation set 1 is 0.907
Accuracy for validation set 2 is 0.908
Accuracy for validation set 3 is 0.911
The highest accuracy is 0.911, from model 3


In [63]:
import pickle
Model_file = './models/num_of_estimators={}.pkl'.format(num_of_estimators)
pickle.dump(models[best_model_num], open(Model_file, 'wb'))

In [10]:
# for output in outputs:
#     evaluation = model_evals[output]
#     plt.figure(figsize=(8, 4), dpi=300)
#     ax = plt.boxplot(evaluation[1].values, vert=False)
#     ax = plt.title('Percent Error of Model for Predicting {}'.format(output), size=20)
#     ax = plt.xlabel('Percent error (%)', size=18)
#     ax = plt.ylabel('Model number', size=18)
#     ax = plt.xlim(x_limit)
#     ax = plt.xticks(size=18)
#     ax = plt.yticks(size=18)
#     plt.show()

In [11]:
# def cdf_func(data, bins=200):
#     hist_data, bin_edges = np.histogram(data, bins=bins, density=True)
#     cdf_bins = np.cumsum(hist_data * np.diff(bin_edges))
#     cdf_bins = np.insert(cdf_bins, 0, 0)
#     cdf = scipy.interpolate.interp1d(bin_edges, cdf_bins, bounds_error=False, fill_value=-1)
#     return cdf

In [12]:
# for output in outputs:
#     best_model_num, evaluation = model_evals[output]
#     best_evaluation = evaluation[best_model_num]
#     x_range = np.linspace(np.min(best_evaluation), np.max(best_evaluation), 1000)

#     fig, ax1 = plt.subplots(figsize=(8,6), dpi=300)

#     ax1.hist(best_evaluation, bins=100, density=True, color='#F25F5C')
#     ax1.tick_params(axis='y', labelcolor='#F25F5C')
#     ax1.set_ylabel('Probability', size=18, color='#F25F5C')
#     ax1.set_xlabel('Percent error (%)', size=18)
#     ax1.set_title('Percent error Distribution, {}'.format(output), size=20)
#     ax1.tick_params(labelsize=18)
#     ax1.set_xlim(x_limit[0], x_limit[1])
#     ax1.set_ylim(y_limit[0], y_limit[1])

#     ax2 = ax1.twinx()
#     ax2.tick_params(axis='y', labelcolor='#247BA0')
#     ax2.set_ylabel('Cumulative Density Function', size=18, color='#247BA0')
#     ax2.tick_params(labelsize=18)

#     ax2.plot(x_range, cdf_func(best_evaluation, 100)(x_range), label='Measured CDF from sample', color='#247BA0')